In [2]:
#Import different libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle


: 

In [57]:
#Import paths
BASE_PATH = "../"
DATA = os.path.join(BASE_PATH, "data")
DATA_TRAIN = os.path.join(DATA, "train")
DATA_VALIDATION = os.path.join(DATA, "validation")

In [59]:
#Parameters setup
img_height, img_width = 150, 150
batch_size = 32
epochs = 30

In [61]:
# Préparer les générateurs de données avec une division entraînement/validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% des données seront utilisées pour la validation
)

train_generator = train_datagen.flow_from_directory(
    DATA_TRAIN,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Utiliser les 80% pour l'entraînement
)

validation_generator = train_datagen.flow_from_directory(
    DATA_TRAIN,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Utiliser les 20% pour la validation
)

# Afficher les classes
print("Classes trouvées :", train_generator.class_indices)

# Afficher le nombre d'images dans chaque générateur
print(f"Nombre d'images d'entraînement : {train_generator.samples}")
print(f"Nombre d'images de validation : {validation_generator.samples}")

Found 16933 images belonging to 4 classes.
Found 4232 images belonging to 4 classes.
Classes trouvées : {'covid': 0, 'lung_opacity': 1, 'normal': 2, 'pneumonia': 3}
Nombre d'images d'entraînement : 16933
Nombre d'images de validation : 4232


In [62]:
#Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
# Définir les callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True)
]

In [65]:
# Entraîner le modèle
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks
)

Epoch 1/20


c:\Users\Raph\miniconda3\envs\pulmoScan4\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


530/530 ━━━━━━━━━━━━━━━━━━━━ 157s 292ms/step - accuracy: 0.5872 - loss: 1.0057 - val_accuracy: 0.7200 - val_loss: 0.6818
Epoch 2/20
530/530 ━━━━━━━━━━━━━━━━━━━━ 157s 294ms/step - accuracy: 0.7362 - loss: 0.6648 - val_accuracy: 0.7890 - val_loss: 0.5464
Epoch 3/20
530/530 ━━━━━━━━━━━━━━━━━━━━ 157s 295ms/step - accuracy: 0.7692 - loss: 0.5818 - val_accuracy: 0.8006 - val_loss: 0.5214
Epoch 4/20
530/530 ━━━━━━━━━━━━━━━━━━━━ 176s 331ms/step - accuracy: 0.7980 - loss: 0.5198 - val_accuracy: 0.8159 - val_loss: 0.4779
Epoch 5/20
530/530 ━━━━━━━━━━━━━━━━━━━━ 163s 306ms/step - accuracy: 0.8129 - loss: 0.4729 - val_accuracy: 0.8459 - val_loss: 0.4357
Epoch 6/20
530/530 ━━━━━━━━━━━━━━━━━━━━ 158s 297ms/step - accuracy: 0.8299 - loss: 0.4488 - val_accuracy: 0.8315 - val_loss: 0.4422
Epoch 7/20
530/530 ━━━━━━━━━━━━━━━━━━━━ 157s 294ms/step - accuracy: 0.8348 - loss: 0.4271 - val_accuracy: 0.8495 - val_loss: 0.4110
Epoch 8/20
530/530 ━━━━━━━━━━━━━━━━━━━━ 158s 297ms/step - accuracy: 0.8531 - loss: 0.40

In [66]:

# Évaluer le modèle
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {accuracy * 100:.2f}%")

133/133 ━━━━━━━━━━━━━━━━━━━━ 21s 156ms/step - accuracy: 0.8803 - loss: 0.3478
Validation accuracy: 88.09%


In [67]:
# Sauvegarder le modèle avec pickle
model.save('cnn_model.h5')

In [ ]:
# Convertir le modèle en pickle
with open('cnn_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Visualiser les courbes d'entraînement
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()